In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# Weight 초기화 함수
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

# CNN 모델 함수
def cnn_model(X, W1, W2, W3, W4, WO, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME'))          # l1a shape(?, 28, 28, 32)
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # l1 shape=(?, 14, 14, 32)
    l1 = tf.nn.dropout(l1, p_keep_conv)
    
    l2a = tf.nn.relu(tf.nn.conv2d(l1, W2, strides=[1, 1, 1, 1], padding='SAME'))        # l2a shape=(? ,14, 14, 64)
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # l2 shape=(?, 7, 7, 64)
    l2 = tf.nn.dropout(l2, p_keep_conv)
    
    l3a = tf.nn.relu(tf.nn.conv2d(l2, W3, strides=[1, 1, 1, 1], padding='SAME'))        # l3a shape=(? ,7, 7, 128)
    l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  # l3 shape=(?, 4, 4, 128)
    l3 = tf.nn.dropout(l3, p_keep_conv)
    
    l3 = tf.reshape(l3, [-1, W4.get_shape().as_list()[0]]) # reshpe to (?, 2048)
    l3 = tf.nn.dropout(l3, p_keep_conv)
    
    l4 = tf.nn.relu(tf.matmul(l3, W4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)
    
    pyx = tf.matmul(l4, WO)
    return pyx

In [ ]:
#Check out https://www.tensorflow.org/get_started/mnist/beginners for more information about the mnist dataset
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28, 1) # 28x28x1 input image
teX = teX.reshape(-1, 28, 28, 1) # 28x28x1 input image

# Input placeholder
X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])

# Filter
W1 = init_weights([3, 3, 1, 32])       # 3x3x1 conv, 32 outputs
W2 = init_weights([3, 3, 32, 64])      # 3x3x32 conv, 64 outputs
W3 = init_weights([3, 3, 64, 128])     # 3x3x64 conv, 128 outputs
W4 = init_weights([128 * 4 * 4, 625])  # Fully connected 128 * 4 * 4 inputs, 625 outputs 
WO = init_weights([625, 10])           # Fully connected 625 inputs, 10 outputs(labels 0 ~ 9)

# Dropout parameter placeholder
p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = cnn_model(X, W1, W2, W3, W4, WO, p_keep_conv, p_keep_hidden)

# Define cost & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict = tf.argmax(py_x, 1)

# Parameters
batch_size = 128
test_size = 256

with tf.Session() as sess:
    # Initialize TensorFlow Variables
    sess.run(tf.global_variables_initializer())
    
    for step in range(100):
        training_batch = zip(range(0, len(trX), batch_size), range(batch_size, len(trX)+1, batch_size))
        
        for start, end in training_batch:
            sess.run(train, feed_dict={X: trX[start:end], Y: trY[start:end], p_keep_conv: 0.8, p_keep_hidden: 0.5})
            
        test_indices = np.arange(len(teX)) # Get A test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]
            
        print(step, np.mean(np.argmax(teY[test_indices], axis=1) ==
                                         sess.run(predict, feed_dict={X: teX[test_indices],
                                                                    Y: teY[test_indices],
                                                                    p_keep_conv: 1.0,
                                                                    p_keep_hidden: 1.0})))